# variant calling tutorials: from large vcf to de novo mutations

this tutorial is under /home/hq2130/tutorials/vcf

## We need to describe mappability filter and anything specific to WGS analysis. 

# split large vcf into vcf trios with only de novo variants

In [16]:
import gzip
import os

In [7]:
vcf_name = 'cmbgvcf.rawvariants.recalibrated.vcf.hg19_multianno.vcf.gz'
ped = '../src/CHD_MedExomeKit.ped'

In [13]:
def check_denovo(trio):
    proband= trio[0].split(':')
    Father = trio[1].split(':')
    Mother = trio[2].split(':')
    proband_GT = proband[0]
    Father_GT = Father[0]
    Mother_GT = Mother[0]
    if proband_GT in {'./.','0/0'}: #proband genotype missing or not denovo
        return False
    else:#GT:AD:DP:GQ:PL    0/1:16,9:.:99:205,0,501
        if Father_GT == '0/0' and Mother_GT == '0/0':
            return True
        else:
            return False

In [14]:

with gzip.open(vcf_name, 'rt') as f:
    for line in f: 
        if line[:6]=='#CHROM': 
            samples = line.split()
            break
            
vcfid2sample = {} # if we want to convert vcf id to different names such as SRR001 to 1-0001
family2id = {}
with open('../src/CHD_MedExomeKit.ped') as f:
    head = f.readline()
    for line in f:
        family, proband, father, mother = line.strip().split('\t')[:4]
        if proband in samples and father in samples and mother in samples:
            
            family2id[family] = [proband, father, mother]
family2id

{'1-04687': ['1-04687', '1-04687-01', '1-04687-02'],
 '1-07134': ['1-07134', '1-07134-01', '1-07134-02']}

In [15]:
with gzip.open(vcf_name, 'rt') as f:
    head = []
    ChrCount=0
    for line in f:
        if line.startswith('#'):
            if line.startswith('#CHROM'):
                vcf_samples = line.strip().split()
                sample2idx = {}
                for idx, sample in enumerate(vcf_samples):
                    sample = vcfid2sample.get(sample, sample)
                    sample2idx[sample] = idx
                vcf_files = []
                for proband in family2id:
                    proband, father, mother = family2id[proband]
                    if proband in sample2idx and father in sample2idx and mother in sample2idx:
                        filename = 'vcf_denovo/' + '_'.join([proband, father, mother]) + '.vcf'
                        vcf_files.append(open(filename, 'wt'))
                n = len(vcf_files)
                for i in range(n):
                    proband, father, mother = vcf_files[i].name.split('/')[-1].split('.vcf')[0].split('_')
                    vcf_files[i].write(''.join(head))
                    vcf_files[i].write('\t'.join(line.strip().split('\t')[:9] + [proband, father, mother])+'\n')  
            else:
                head.append(line)
        else:
            data = line.strip().split('\t')
            # verbose  
            ChrPresent = data[0]
            if ChrCount != ChrPresent:
                print("Chromosome "+str(ChrPresent))
                ChrCount=ChrPresent
                
            for i in range(n):
                proband, father, mother = vcf_files[i].name.split('/')[-1].split('.vcf')[0].split('_')
                trio = [data[sample2idx[proband]], data[sample2idx[father]], data[sample2idx[mother]]]
                if check_denovo(trio):
            
                    # filter AC larger than 60 
                    INFOcolumnList=data[7].split(";")
                    INFOdict={}
                    for element in INFOcolumnList:
                        if '=' in element:
                            FieldName,FieldValue=element.split('=',1)
                            INFOdict[FieldName]=FieldValue
                            if FieldName == 'AC':
                                break
                    AC = sum(map(int, INFOdict['AC'].split(',')))
                    if AC >= 60:
                        continue
                
            
                    vcf_files[i].write('\t'.join(data[:9]+trio)+'\n')
            
for fh in vcf_files:
    fh.close()

Chromosome 1
Chromosome 2
Chromosome 3
Chromosome 4
Chromosome 5
Chromosome 6
Chromosome 7
Chromosome 8
Chromosome 9
Chromosome 10
Chromosome 11
Chromosome 12
Chromosome 13
Chromosome 14
Chromosome 15
Chromosome 16
Chromosome 17
Chromosome 18
Chromosome 19
Chromosome 20
Chromosome 21
Chromosome 22
Chromosome X
Chromosome Y


# aggregate denovo trio vcfs into one csv

In [19]:
from collections import defaultdict
import csv

def info_list(INFOstring, head):    
    INFOcolumnList=INFOstring.split(";")
    INFOdict={}
    for element in INFOcolumnList:
        if '=' in element:
            FieldName,FieldValue=element.split('=',1)
            INFOdict[FieldName]=FieldValue           
    result = []
    for e in head:
        result.append(INFOdict.get(e,'NA'))
    return result



denovo_dir = 'vcf_denovo/'

head = set()
mutation = {}
mutation_counts = defaultdict(lambda: [0, 0])
for e in os.listdir(denovo_dir):
    if e.endswith('.vcf'):
        with open(denovo_dir +'/'+ e) as f1:
            proband, father_ID, mother_ID= e.rsplit('.vcf')[0].split('_')
            
            for line in f1:
                if line[0] =='#':
                    if line.startswith('##INFO=<ID='):
                        head.add(line.split('=')[2].split(',')[0])
                else:
                    info = line.strip().split()
                    chrom = info[0]
                    pos = info[1]
                    INFOstring = info[7]
                    format = info[8]

                    variant_id = '_'.join([proband, chrom, pos])
                    

                    proband_geno = info[9].split(':')[0]
                    proband_info = info[9]
                    Father_geno = info[10].split(':')[0]
                    Father_info = info[10]
                    Mother_geno = info[11].split(':')[0]
                    Mother_info = info[11]
                    parents_list = father_ID + '(' + Father_info + ')' + ',' + mother_ID +'('+ Mother_info +')'
                    
                    variant_id_2 = '_'.join([chrom, pos])
                    mutation_counts[variant_id_2][0] += 1

                    if variant_id in mutation:
                        print('same varaint in same proband again:' , variant_id)
                    else:
                        mutation[variant_id] = [format, proband+'('+ proband_info +')', parents_list] + info[:8]
head = list(head)
head.sort()
with open('csv_denovo/test0713.csv','wt') as f:
    w = csv.writer(f)
    w.writerow(['CHROM','POS', 'ID', 'REF', 'ALT', 'QUAL','FILTER','FORMAT', 
                'proband', 'parents', 'het', 'homo'] + head)
    for variant_id, value in mutation.items():
        variant_id_2 = '_'.join(variant_id.split('_')[-2:])
        content =  value[3:-1] + value[:3] + mutation_counts[variant_id_2] + info_list(value[-1], head)
        w.writerow(content)

# filter for high confident denovo mutations

In [20]:
import datetime
import yaml
import csv

In [23]:
now = datetime.datetime.now().strftime("%Y-%m-%d")
with open('../src/denovo_coding_soft.yml', 'r') as ymlfile:
    cfg = yaml.load(ymlfile)

csv_name = 'csv_denovo/test0713.csv'

with open(csv_name.rsplit('/', 1)[0] + '/filter_parameter_' + now +'.txt', 'wt') as fw:
    for section in cfg:
        for key, value in cfg[section].items():
            print(key, value)
            fw.write('    %s %s: %s\n' % (type(value), key, value))

exon True
exon_flag ['splicing', 'exonic', 'exonic,splicing']
max_cohort_AC 6
max_AC_total 6
max_population_AF {'ESP': 0.001, '1KG': 0.001, 'ExAC': 0.001}
excluded_gene ['MUC', 'HLA']
excluded_chrom ['GL']
max_Multiallelic 3
max_seqdup 0.99
min_proband_AD 5
min_proband_PL 60
min_proband_alt_freq_tier1 0.2
min_proband_alt_freq_tier2 0.2
min_parents_DP 10
min_parents_GQ 30
max_parents_alt_freq 0.05
max_FS 25
min_QD 2
max_FS 25
min_QD 1
min_ReadPosRankSum -3


In [24]:
# VarFunc exonic

def info_filter(variants, snps, parameter):
    '''
    functions that flag the variant of FS, QD
    '''
    p = True

    # freq filter
    ExACfreq = max(map(float,[0] + [rate for rate in variants.get('ExAC_ALL','0').split(',') if rate not in {'NA','.'} ]))
    
    if ExACfreq > parameter['gene_filter']['max_population_AF']['ExAC']:
        p = False

    # exon filter
    if parameter['gene_filter']['exon']:
        if variants['Func.refGene'] not in parameter['gene_filter']['exon_flag']:
            p = False

    if variants['QD'] == 'NA':
        variants['QD'] = 100
    if snps:
        if float(variants['FS']) > parameter['snps']['max_FS']:
            p = False
        if variants['QD'] == 'NA' or float(variants['QD']) < parameter['snps']['min_QD']:
            p = False 
    else:
        if float(variants['FS']) > parameter['indels']['max_FS']:
            p = False
        if float(variants['QD']) < parameter['indels']['min_QD']:
            p = False   
        if variants['ReadPosRankSum'] != 'NA' and float(variants['ReadPosRankSum']) < parameter['indels']['min_ReadPosRankSum']:
            p = False   

    return p

def variants_filter(filename, parameter):
    with open(filename,'rt') as denovo_f:
        denovo_r = csv.reader(denovo_f)
        head = next(denovo_r)

        denovo_snp, denovo_indel =[], []
        
        denovo_write= open(filename.rsplit('_', 1)[0] + '_filtered_'+now+'.csv','wt')
        w= csv.writer(denovo_write)
        w.writerow(head) 

        denovo2_write= open(filename.rsplit('_', 1)[0] + '_tie2.csv','wt')
        w2 = csv.writer(denovo2_write)
        w2.writerow(head) 

        
        idx = dict(zip(head, range(len(head))))
        for line in denovo_r:

            variant_pass = True
            variant = dict(zip(head, line))
            chrom = variant['CHROM']
            pos = variant['POS']
            
            
            ref = variant['REF']
            alt = variant['ALT']
            format = variant['FORMAT']
            probands = variant['proband'].split(';')
            parents = variant['parents'].split(';')
            AC = variant['AC'].split(',') 
            
            # remove multi-allel sites
            if len(alt.split(',')) > parameter['gene_filter']['max_Multiallelic'] : 
                variant_pass = False
            #print('0', variant_pass)
            # remove know problem genes
            for e in parameter['gene_filter']['excluded_chrom']:
                if e in chrom:
                    variant_pass = False

            for e in parameter['gene_filter']['excluded_gene']:
                if variant['Gene.refGene'].startswith(e):
                    variant_pass = False

            # remove high segdup score  
            segdup = 0 if variant.get('genomicSuperDups') in {'NA','.'} else float(variant.get('genomicSuperDups',':0').split(':')[1].split('-')[0])
            if segdup > parameter['gene_filter']['max_seqdup']: 
                variant_pass = False
            #print('a', variant_pass)
            
            for i in range(len(probands)) :
                newline = list(line)
                proband = probands[i]
                newline[idx['proband']] = proband
                parent = parents[i] 
                newline[idx['parents']] = parent
                ID, geno = proband.split('(')
                GT = geno[:-1].split(':')[0]
                _, alt_idx = map(int,GT.split('/'))
                newline[idx['ALT']] = alt.split(',')[alt_idx-1]   


                real_AC = int(AC[alt_idx-1])
                if real_AC > parameter['gene_filter']['max_cohort_AC']:
                    variant_pass = False
                total_AC = sum(map(int, AC))
                if total_AC > 2 * parameter['gene_filter']['max_cohort_AC']:
                    variant_pass = False

                snp = len(newline[idx['ALT']]) == len(ref) and newline[idx['ALT']] != '*'


                
                if not info_filter(variant, snp, parameter):
                    variant_pass = False
                
                #print('b', variant_pass)
                variant_id = '_'.join([ID,chrom,pos])

                # change varclass format
                varclass = variant['ExonicFunc.refGene']
                if len(variant['ExonicFunc.refGene'].split(',')) > 1:
                    varclass = variant['ExonicFunc.refGene'].split(',')[alt_idx-1]
                    newline[idx['ExonicFunc.refGene']] = varclass
               
                #print('c', variant_pass, reads_filter(format, proband, parent, parameter))
                if snp: 
                    if variant_pass and reads_filter(format, proband, parent, parameter):
                        denovo_snp.append(variant_id)
                        w.writerow(newline)  
                    elif variant_pass:# and check_damage(variant, varclass):
                        w2.writerow(newline)  

                else:
                    if variant_pass and reads_filter(format, proband, parent, parameter):
                        denovo_indel.append( variant_id)
                        w.writerow(newline)   
                    elif variant_pass:# and check_damage(variant, varclass):
                        w2.writerow(newline)              

        denovo_write.close()
        denovo2_write.close()
        denovo_f.close()
    return denovo_snp, denovo_indel

def check_damage(variants, varclass):
    '''
    function return if a variant if damage 
    '''
    damage = False
    damage_key = {'frameshift','stop','splicing'}
    for key in damage_key:
        if varclass.startswith(key): 
            damage = True
    
    if 'nonsynonymous_SNV' in  varclass:
        if 'D' in variants['MetaSVM_pred']:
            damage = True
        elif 'D' in variants['Polyphen2_HDIV_pred']:
            cadd = max(map(float,[ rate  for rate in variants['CADD_phred'].split(',') if rate != '.' ]))
            if cadd >= 15:
                damage = True
    return damage


def get_dp(dp, ad):
    '''reformat dp
    '''
    if dp == '.':
        dp = sum(map(int, ad.split(',')))
    dp = max([1, float(dp)])
    return dp



def reads_filter(format, proband, parents, parameter): # R,A used for checking indel AC is [1,1]
    reads_pass = True
    ID, geno =  proband.split('(')
    father,mother = parents.split('),')
    fatherID,fathergeno = father.split('(')
    motherID,mothergeno = mother[:-1].split('(')

    #print parents
    proband_format = dict(zip(format.split(':'), geno[:-1].split(':')))
    GT,AD,DP,GQ,PL =  proband_format['GT'], proband_format['AD'], proband_format['DP'], proband_format['GQ'], proband_format['PL']
    father_format = dict(zip(format.split(':'), fathergeno.split(':')))
    fGT,fAD,fDP,fGQ,fPL =  father_format['GT'], father_format['AD'], father_format['DP'], father_format['GQ'], father_format['PL']
    mother_format = dict(zip(format.split(':'), mothergeno[:-1].split(':')))
    mGT,mAD,mDP,mGQ,mPL  =  mother_format['GT'], mother_format['AD'], mother_format['DP'], mother_format['GQ'], mother_format['PL']

        
    ref_idx, alt_idx = map(int,GT.split('/'))
    
    # parents filter
    if  mDP == '.':
        mDP = max([1, sum(map(int, mAD.split(',')))])
    if  fDP == '.':
        fDP = max([1, sum(map(int, fAD.split(',')))])
    if  DP in {'.','0'}:
        if DP == '.':
            DP = sum(map(int, AD.split(',')))
        else:
            print (proband, DP, 'proband dp . ??')
            DP = 1  
                
    if float(mDP) < 1:
        mDP = 1
    if float(fDP) < 1:
        fDP = 1
    Fmaf = int(fAD.split(',')[alt_idx])/float(fDP)
    Mmaf = int(mAD.split(',')[alt_idx])/float(mDP) 

     # parents filter
    #print('xa', reads_pass, max(Fmaf, Mmaf), parameter['reads_filter']['max_parents_alt_freq'])
    if max(Fmaf, Mmaf) > parameter['reads_filter']['max_parents_alt_freq']:
        reads_pass = False
    if min(int(mDP), int(fDP)) < parameter['reads_filter']['min_parents_DP']:
        reads_pass = False
    if min(int(mGQ), int(fGQ)) < parameter['reads_filter']['min_parents_GQ']:
        reads_pass = False
    # prband filter
    proband_AD = list(map(int,AD.split(',')))
    proband_DP = max([1, float(DP)])
    proband_PL = int(PL.split(',')[0])
    #print proband_DP, proband
    proband_maf = proband_AD[alt_idx]/proband_DP
    
   # print('x', reads_pass)

    if proband_PL < parameter['reads_filter']['min_proband_PL']: #require proband_PL of 0/0 larger than p, which is probably of 0/0 is 10^-p
        reads_pass = False
    
    #print('x', reads_pass, proband_AD[alt_idx], parameter['reads_filter']['min_proband_AD'])
    
    if proband_AD[alt_idx] <  parameter['reads_filter']['min_proband_AD']:
        reads_pass = False
    
    if proband_AD[alt_idx] < 10:
        if proband_maf < parameter['reads_filter']['min_proband_alt_freq_tier1']:
            reads_pass = False
    else:
        if proband_maf < parameter['reads_filter']['min_proband_alt_freq_tier2']:
            reads_pass = False
    return  reads_pass


In [25]:
denovo_snp, denovo_indel = variants_filter(csv_name, cfg)
print ('%s denovo snp, %s denovo indels' % (len(denovo_snp), len(denovo_indel)))

2 denovo snp, 1 denovo indels
